# Remote inspection with machine learning at the edge

This notebook builds a machine learning (ML) model to perform remote inspection at the edge. The model is developed to run on a device at the edge. The model makes predictions based on thermal images captured at the edge by the device. The model is built using the Yolov4 library.

## Yolov4 Pytorch 1.7 for Edge Devices with Amazon SageMaker

Amazon SageMaker is a fully managed machine learning service. With SageMaker, data scientists and developers can quickly and easily build and train machine learning models, and then directly deploy them into a production-ready hosted environment. It provides an integrated Jupyter authoring notebook instance for easy access to your data sources for exploration and analysis, so you don't have to manage servers. It also provides common machine learning algorithms that are optimized to run efficiently against extremely large data in a distributed environment. With native support for bring-your-own-algorithms and frameworks, SageMaker offers flexible distributed training options that adjust to your specific workflows.

SageMaker also offers capabilities to prepare models for deployment at the edge. [SageMaker Neo](https://docs.aws.amazon.com/sagemaker/latest/dg/neo.html) is a capability of Amazon SageMaker that enables machine learning models to train once and run anywhere in the cloud and at the edge and [Amazon SageMaker Edge Manager](https://docs.aws.amazon.com/sagemaker/latest/dg/edge.html) provides model management for edge devices so you can optimize, secure, monitor, and maintain machine learning models on fleets of edge devices such as smart cameras, robots, personal computers, and mobile devices.

In this notebook we'll train a [**Yolov4**](https://github.com/WongKinYiu/PyTorch_YOLOv4) model on Pytorch using Amazon SageMaker to draw bounding boxes around images and then, compile and package it so that it can be deployed on an edge device(in this case, a [Jetson Xavier](https://developer.nvidia.com/jetpack-sdk-441-archive)).

## 1) Pre-requisites

Let us start with setting up the pre-requisites for this notebook. First, we will sagemaker and other related libs and then set up the role and buckets and some variables. Note that, we are also specifying the size of the image and also the model size taken as Yolov4s where s stand for small. Check out the [github doc of yolov4](https://github.com/WongKinYiu/PyTorch_YOLOv4) to understand how model sizes differ.

In [ ]:
import sagemaker
import numpy as np
import glob
import os
from sagemaker.pytorch.estimator import PyTorch

from matplotlib import pyplot as plt  # for image display
import matplotlib.patches as patches  # for image display
from PIL import Image as pil_image    # PIL is the Python Imaging Library for image display
import random                         # for image display
from pathlib import Path              # for image display

role = sagemaker.get_execution_role()
sagemaker_session=sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
img_size=640
model_type='tiny' # tiny or ''
model_name='yolov4'if model_type=='' else f"yolov4-{model_type}"

### 1.1) Download a public implementation of Yolov4 for Pytorch

Now, we will download the PyTorch implementation of Yolov4 from this [repository](https://github.com/WongKinYiu/PyTorch_YOLOv4) which is authored by Wong Kin Yiu. We will place it in a local directory `yolov4` and apply a patch for cuda

In [ ]:
if not os.path.isdir('yolov4'):
    !git clone https://github.com/WongKinYiu/PyTorch_YOLOv4 yolov4
    !cd yolov4 && git checkout 3c42cbd1b0fa28ad19436d01e0e240404463ff80 && git apply ../mish.patch
    !echo 'tensorboard' > yolov4/requirements.txt

### 1.2) Image display constants

In [ ]:
image_path = './panelscoco/images/'
image_extension = '.jpg'
label_path = './panelscoco/labels/'
label_extension = '.txt'

### 1.3) Function to display image with all bounding boxes

In [ ]:
def display_image_with_bounding_boxes (image, boxes):
    # function to display an image with its bounding boxes
    #
    # image is a Python Image Library image object holding image
    # boxes is a np.array holding bounding boxes
    #    in COCO format bounding box annotation
    #    [class, x_center, y_center, width, height]
    #    in dimensions as a fraction of the whole

    def add_bounding_box_to_image (axis, box, width, height):
        # function to add a single bounding box to an image
        #
        # axis is MatLibPlot axis object
        # box is np array holding dimensions for a single bounding box
        #    in COCO format bounding box annotation
        #    [class, x_center, y_center, width, height]
        #    in dimensions as a fraction of the whole
        # width is image width in pixels
        # height is image height in pixels
        
        # Create a Rectangle patch for the box
        rect = patches.Rectangle(((box[1]-box[3])*width, (box[2]-box[4])*height),
                                 box[3]*width,
                                 box[4]*height,
                                 edgecolor='r',
                                 facecolor='none')
        axis.add_patch(rect)
        
    # get image width & height
    width, height = image.size

    # Create the figure and axis
    fig, ax = plt.subplots()
    
    # set the image
    ax.imshow(np.array(image))    # convert image to np.array for use by MatLibPlot imshow
    
    # loop through boxes to add each bounding box to the image
    # get number of boxes
    number_of_boxes = boxes.shape[0]    # index 0 returns first dimension, index 1 returns second dimension, etc.
    for counter in range(number_of_boxes):    # loops from 0 to (number_of_boxes - 1)
        # get a single box from boxes
        box = boxes [counter]
        # add the bounding box to the image
        add_bounding_box_to_image (ax, box, width, height)
    
    # show the image with the bounding boxes
    plt.show()

## 2) Data engineering (inspection, preparation, and analysis)

### 2.1) Data inspection  

#### 2.1.1) Label format

Lets look at the label format used for the dataset. It should follow the same standard as COCO.

If you need an example of the COCO format standard, a sample dataset is available [coco128](https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip). Use this code to download it.

```
if not os.path.exists('coco128'):
    !wget -q https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip
    !unzip -q coco128.zip && rm -f coco128.zip
```

In [ ]:
print('BBoxes annotation')
print('class x_center y_center width height')
!head panelscoco/labels/0_1-0_thermal.txt

#### 2.1.2) Randomly select four images and display them with there bounding boxes

In [ ]:
for counter in range(4):
    # randomly get a filename with extension but without the path
    image_filename = random.choice(os.listdir(image_path))
    # get the stem of the filename
    file_stem = Path(image_filename).stem
    print("stem", file_stem)
    image_file = image_path + file_stem + image_extension    # create the full path to the image 
    label_file = label_path + file_stem + label_extension    # create the full path to the bounding box labels for the image
    
    if Path(label_path + file_stem + label_extension).stat().st_size > 0:    # ignore if the labels are empty
        # image as a np.array holding image
        image = pil_image.open(image_file)
        # boxes as a np.array holding bounding boxes
        boxes = np.genfromtxt(label_file, delimiter=' ')
        display_image_with_bounding_boxes (image, boxes)
    else:
        print("... has no bounding box labels")

### 2.2) Data preparation  

#### 2.2.1) Upload the dataset to S3

The dataset is uploaded into an S3 bucket. The S3 bucket was created outside of this notebook. 

The training and validation dataset S3 locations are defined here.

In [ ]:
prefix='data/panelscoco'
!rm -f panelscoco/labels/train2017.cache
train_path = sagemaker_session.upload_data('panelscoco', key_prefix=f'{prefix}/train')
val_path = sagemaker_session.upload_data('panelscoco', key_prefix=f'{prefix}/val')
print(train_path, val_path)

In [ ]:
!aws s3 ls $train_path/images/

## 3) Train the model

### 3.1) Prepare a Python script that will be the entrypoint of the training process

Now, we will create a training script to train the Yolov4 model. The training script will wrap the original training scripts and expose the parameters to SageMaker Estimator. The script accepts different arguments which will control the training process. In this specific case number of classes=1 (CELL)

In [ ]:
%%writefile yolov4/sagemaker_train.py
import sys
import subprocess
## We need to remove smdebug to avoid the Hook bug https://github.com/awslabs/sagemaker-debugger/issues/401
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "smdebug"])
import os
import yaml
import argparse
import torch
import shutil
import urllib
from models.models import Darknet

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--num-classes', type=int, default=1, help='Number of classes')
    parser.add_argument('--img-size', type=int, default=640, help='Size of the image')
    parser.add_argument('--epochs', type=int, default=1, help='Number of epochs')
    parser.add_argument('--batch-size', type=int, default=16, help='Batch size')
    parser.add_argument('--adam', action='store_true', help='use torch.optim.Adam() optimizer')
    parser.add_argument('--pretrained', action='store_true', help='use pretrained model')
    
    parser.add_argument('--model-dir', type=str, default=os.environ["SM_MODEL_DIR"], help='Trained model dir')
    parser.add_argument('--train', type=str, default=os.environ["SM_CHANNEL_TRAIN"], help='Train path')
    parser.add_argument('--train-suffix', type=str, default='', help='Train path suffix')
    parser.add_argument('--validation', type=str, default=os.environ["SM_CHANNEL_VALIDATION"], help='Validation path')
    parser.add_argument('--validation-suffix', type=str, default='', help='Validation path suffix')
    
    parser.add_argument('--model-type', type=str, choices=['', 'tiny'], default="", help='Model type')
    
    # hyperparameters
    with open('data/hyp.scratch.yaml', 'r') as f:
        hyperparams = yaml.load(f, Loader=yaml.FullLoader)    
    for k,v in hyperparams.items():
        parser.add_argument(f"--{k.replace('_', '-')}", type=float, default=v)
    
    args,unknown = parser.parse_known_args()
    
    base_path=os.path.dirname(__file__)
    project_dir = os.environ["SM_OUTPUT_DATA_DIR"]

    # prepare the hyperparameters metadat
    with open(os.path.join(base_path,'data', 'hyp.custom.yaml'), 'w' ) as y:
        y.write(yaml.dump({h:vars(args)[h] for h in hyperparams.keys()}))

    # prepare the training data metadata
    with open(os.path.join(base_path,'data', 'custom.yaml'), 'w') as y:
        y.write(yaml.dump({            
            'names': [f'class_{i}' for i in range(args.num_classes)],
            'train': os.path.join(args.train, args.train_suffix),
            'val': os.path.join(args.validation, args.validation_suffix),
            'nc': args.num_classes
        }))
    model_name = "yolov4" if len(args.model_type) == 0 else f"yolov4-{args.model_type}"
    # run the training script
    weights_file=''
    if args.pretrained:
        weights_file = f'weights/{model_name}.weights'
        urllib.request.urlretrieve(
            f'https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/{model_name}.weights',
            weights_file
        )

    train_cmd = [
        sys.executable, os.path.join(base_path,'train.py'),
        "--data", "custom.yaml",
        "--hyp", "hyp.custom.yaml",
        "--cfg", f"cfg/{model_name}.cfg",
        "--img", str(args.img_size),
        "--batch", str(args.batch_size),
        "--epochs", str(args.epochs),
        "--logdir", project_dir,
        "--weights", weights_file
    ]
    if args.adam: train_cmd.append("--adam")
    subprocess.check_call(train_cmd)
        
    # tracing and saving the model
    inp = torch.rand(1, 3, args.img_size, args.img_size).cpu()
    ckpt = torch.load(os.path.join(project_dir, 'exp0', 'weights', 'best.pt'), map_location='cpu')
    model = Darknet(f"cfg/{model_name}.cfg").cpu()
    # do not invoke .eval(). we don't need the detection layer
    model.load_state_dict(ckpt['model'], strict=False)
    p = model(inp)
    model_trace = torch.jit.trace(model, inp)
    model_trace.save(os.path.join(args.model_dir, 'model.pth'))

### 3.2) Prepare the SageMaker Estimator to train the model

Now it's time to create an Estimater to train the model with the training script created earlier. We are using Pytorch estimator and supplying other arguments in the estimator. Note that we are supplying the `source_dir` so that sagemaker can pick up the training script and other related files from there. Once the estimator is ready, we start the training using the `.fit()` method.

In [ ]:
estimator = PyTorch(
    'sagemaker_train.py',
    source_dir='yolov4',
    framework_version='1.7',
    role=role,
    sagemaker_session=sagemaker_session,
    instance_type='ml.p3.2xlarge',    
    instance_count=1,
    py_version='py3', 
    hyperparameters={
        'epochs': 20, # at least 2 epochs
        'batch-size': 8,
        'lr0': 0.0001,
        
        'pretrained': True, # transfer learning
        'num-classes': 1,
        'img-size': img_size,
        'model-type': model_type,
        
        # the final path needs to point to the images dir
        'train-suffix': 'images',
        'validation-suffix': 'images'
    }
)

### 3.3) Train the model

In [ ]:
estimator.fit({'train': train_path, 'validation': val_path})

#### 3.3.1) Get the location of the trained model

In [ ]:
s3_uri=f'{estimator.output_path}{estimator.latest_training_job.name}/output/model.tar.gz'

## 4) Compile your trained model for the edge device

Once the model has been trained, we need to compile the model using SageMaker Neo. This step is needed to compile the model for the specific hardware on which this model will be deployed. 

In this notebook, we will compile a model for [Jetson Xavier Jetpack 4.4.1](https://developer.nvidia.com/jetpack-sdk-441-archive). 

In case, you want to compile for a different hardware platform, just change the parameters bellow to adjust the target to your own edge device. Also, note, that if you dont have GPU available on the hardware device, then you can comment the `Accelerator` key:value in the `OutputConfig`.

The below cell also calls the `describe_compilation_job` API in a loop to wait for the compilation job to complete. In actual applications, it is advisable to setup a cloudwatch event which can notify OR execute the next steps once the compilation job is complete.

In [ ]:
import time
import boto3
sm_client = boto3.client('sagemaker')
compilation_job_name = f'{model_name}-pytorch-{int(time.time()*1000)}'
sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': s3_uri,
        'DataInputConfig': f'{{"input": [1,3,{img_size},{img_size}]}}',
        'Framework': 'PYTORCH'
    },
    OutputConfig={
        'S3OutputLocation': f's3://{sagemaker_session.default_bucket()}/{model_name}-pytorch/optimized/',
        'TargetPlatform': { 
            'Os': 'LINUX', 
            'Arch': 'X86_64', # change this to X86_64 if you need o leave ARM64 for Jetson
            #'Accelerator': 'NVIDIA'  # comment this if you don't have an Nvidia GPU
        },
        # Comment or change the following line depending on your edge device
        # Jetson Xavier: sm_72; Jetson Nano: sm_53
        #'CompilerOptions': '{"trt-ver": "7.1.3", "cuda-ver": "10.2", "gpu-code": "sm_53"}' # Jetpack 4.4.1 Comment fo Ec2
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 900 }
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)


## 5) Create a SageMaker Edge Manager packaging job

Once the model has been compiled, it is time to create an edge manager packaging job. Packaging job take SageMaker Neo–compiled models and make any changes necessary to deploy the model with the inference engine, Edge Manager agent.

We need to provide the name used for the Neo compilation job, a name for the packaging job, a role ARN, a name for the model, a model version, and the Amazon S3 bucket URI for the output of the packaging job. Note that Edge Manager packaging job names are case-sensitive.


The below cell also calls the `describe_edge_packaging_job` API in a loop to wait for the packaging job to complete. In actual applications, it is advisable to setup a cloudwatch event which can notify OR execute the next steps once the compilation job is complete.

In [ ]:
import time
model_version = '1.0'
edge_packaging_job_name=f'{model_name}-pytorch-{int(time.time()*1000)}'
resp = sm_client.create_edge_packaging_job(
    EdgePackagingJobName=edge_packaging_job_name,
    CompilationJobName=compilation_job_name,
    ModelName=model_name,
    ModelVersion=model_version,
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': f's3://{bucket_name}/{model_name}'
    }
)
while True:
    resp = sm_client.describe_edge_packaging_job(EdgePackagingJobName=edge_packaging_job_name)    
    if resp['EdgePackagingJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['EdgePackagingJobStatus'], compilation_job_name)
        break
    time.sleep(5)

## 6) Pre processing + Post processing code
After compiling your model, it's time to prepare the application that will use it. In the image bellow you can see the operators that are used by the last layer **YOLOLayer**. When in evaluation mode, this layer applies some operations to merge the two outputs of the network and prepare the predictions for the **Non Maximum Suppression**. In training model, you just have the two raw outputs. 

Given we're using a pruned version (training mode) of the network, you need to apply some post processing code to your predictions.

<table style="border: 1px solid black; border-collapse: collapse;" border=3 cellpadding=0 cellspacing=0>
    <tr style="border: 1px solid black;">
        <td style="text-align: center; border-right: 1px solid;" align="center"><b>WITH DETECTION (evaluation mode)</b></td>
        <td style="text-align: center;" align="center"><b>NO DETECTION (training mode)</b></td>
    </tr>    
    <tr style="border: 1px solid black; border-right: 1px solid;">
        <td width="75%" style="border-right: 1px solid;">
            <img src="imgs/yolov4_detection.png"/>
        </td>
        <td>
            <img src="imgs/yolov4_no_detection.png"/>
        </td>
    </tr>
</table>

All the operations required by this process can be found in the script **[Utils](utils.py)**.
#### WARNING: run the next cell and copy the output to your **utils.py** before running the application. It is necessary to adjust the anchors and scores for the correct yolov4 version


Here it is an example of how to use the code:
```Python
import utils

### your code here
### def predict(model, x):...

confidence_treshold=0.1
# Read the image using OpenCV
img = cv2.imread('dog.jpg')
# Convert the image to the expected network input
x = utils.preprocess_img(img, img_size=416)
# Run the model and get the predictions
preds = predict(model, x)
# Convert the predictions into Detections(bounding boxes, scores and class ids)
detections = utils.detect(preds, 0.25, 0.5, True)
# Iterate over the detections and do what you need to do
for top_left_corner,bottom_right_corner, conf, class_id in detections:
    if confidence_treshold < 0.1: continue
    print( f"bbox: {top_left_corner},{bottom_right_corner}, score: {conf}, class_id: {class_id}")
```

In [ ]:
## This code parses the .cfg file of the version you're using and
## Prints the correct anchors/stride. Copy the output to the 'utils.py' file used by your application
import numpy as np

cfg_filename=f'yolov4/cfg/{model_name}.cfg'
yolo_layer=False
yolo_layers = []

for i in open(cfg_filename, 'r').readlines(): # read the .cfg file
    i = i.strip()
    if len(i) == 0 or i.startswith('#'): continue # ignore empty lines and comments
    elif i.startswith('[') and i.endswith(']'): # header
        if i.lower().replace(' ', '') == '[yolo]':
            yolo_layer = True # yolo layer
            yolo_layers.append({})
    elif yolo_layer: # properties of the layer
        k,v = [a.strip() for a in i.split('=')] # split line into key, value
        if k == 'anchors': # parse anchors
            anchors = np.array([int(j.strip()) for j in v.split(',')])
            yolo_layers[-1]['anchors'] = anchors.reshape((len(anchors)//2, 2))
        elif k == 'mask': # parse mask
            yolo_layers[-1]['mask'] = np.array([int(j.strip()) for j in v.split(',')])

stride = [8, 16, 32, 64, 128]  # P3, P4, P5, P6, P7 strides
if model_type == 'tiny':  # P5, P4, P3 strides
    stride = [32, 16, 8]

print("##### Copy the following lines to your util.py")
print(f"anchors = {[l['anchors'][l['mask']].tolist() for l in yolo_layers]}")
print(f"stride = {stride}")

### Done !!

And we are done with all the steps needed to prepare the model for deploying to edge. The model package is avaialble in S3 and can be taken from there to deploy it to edge device. Now you need to move over to your edge device and download and setup edge manager agent(runtime), model and other related artifacts on the device. Please check out the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/edge.html) for detailed steps.